In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from wordcloud import WordCloud, ImageColorGenerator
import jieba
import jieba.analyse

!apt-get install -qq fonts-noto-cjk
!wget -O dict.txt.big https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
!wget -q https://github.com/google/fonts/raw/main/ofl/notosanstc/NotoSansTC-Regular.otf -O /content/NotoSansTC-Regular.otf
!pip install --upgrade Pillow

--2025-07-01 02:08:32--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2025-07-01 02:08:33--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [application/octet-stream]
Saving to: ‘dict.txt.big’

dict.txt.big        100%[===================>]   8.18M  --.-KB/s    in 0.08s   

2025-07-01 02:08:33 (104 MB/s) - ‘dict.txt.big’ saved [8583143/8583143]



In [ ]:
# 定義函數下載搜尋結果前五頁的網頁檔案
def download_first_five_pages():
    directory = "/content/job_listings"
    if not os.path.exists(directory):
        os.makedirs(directory)

    for page_number in range(1, 51):
        html_to_save = f"https://www.104.com.tw/jobs/search/?jobsource=index_s&keyword=n8n&mode=s&page={page_number}&order=15"
        r = requests.get(html_to_save)
        with open(f"{directory}/search_result_page_{page_number}.html", "w") as file:
            file.write(r.text)
        print(f"Downloaded page {page_number} of job listings.")

# 定義函數擷取前五頁搜尋結果的職缺連結
def get_job_url_list() -> list:
    directory = "/content/job_listings"
    job_title_css_selector = "h2 > a"
    job_url_list = []

    for page_number in range(1, 51):
        with open(f"{directory}/search_result_page_{page_number}.html") as file:
            soup = BeautifulSoup(file, 'html.parser')
        job_url_hrefs = [elem.get("href") for elem in soup.select(job_title_css_selector)]
        job_urls = [job_url_href if job_url_href.startswith("https") else f"https:{job_url_href}" for job_url_href in job_url_hrefs if "hotjob_chr" not in job_url_href]
        job_url_list += job_urls
        print(f"Extracted job URLs from page {page_number}.")

    print(f"Total job URLs extracted: {len(job_url_list)}")
    return job_url_list

# 定義函數下載職缺連結的網頁檔案
def download_job_descriptions(job_url_list: list):
    directory = "/content/job_listings/job_descriptions"
    if not os.path.exists(directory):
        os.makedirs(directory)

    for idx, job_url in enumerate(job_url_list):
        r = requests.get(job_url)
        job_url_split = job_url.split("?")[0]
        page_name = job_url_split.split("/")[-1]
        with open(f"{directory}/{page_name}.html", "w") as file:
            file.write(r.text)
        print(f"Downloaded job description {idx + 1} of {len(job_url_list)}")

        # 增加等待時間以降低爬取速度
        time.sleep(1)  # 每爬取一次職缺頁面後等待 2 秒


# 執行函數
download_first_five_pages()
job_url_list = get_job_url_list()
download_job_descriptions(job_url_list)


# 擷取職缺的工作敘述
directory = "/content/job_listings/job_descriptions"
list_dir = os.listdir(directory)
job_titles, employers, job_locations, salaries, job_descriptions = [], [], [], [], []
work_experience_list, education_requirements_list, tools_list, skills_list, other_conditions_list = [], [], [], [], []

for html_file in list_dir:
    with open(f"{directory}/{html_file}") as file:
        soup = BeautifulSoup(file, 'html.parser')

    # 職缺標題
    #job_title_element = soup.select("div.job-header__title > h1")
    job_title_element = soup.select("hgroup > h1, div.job-header__title > h1")
    job_title = job_title_element[0].text.strip() if job_title_element else "N/A"

    # 公司名稱
    employer_element = soup.select("div.job-header__title a[data-gtm-head='公司名稱']")
    if employer_element:
        employer = employer_element[0].text.strip()
    else:
        employer = "N/A"

    # 工作地點
    job_location_element = soup.find("div", class_="job-address")
    if job_location_element and job_location_element.has_attr("addressarea"):
        job_location = job_location_element["addressarea"].strip()
    else:
        job_location = "N/A"

    # 薪資範圍
    salary_element = soup.select("div.list-row__data p.text-primary.font-weight-bold")
    if salary_element:
        salary = salary_element[0].text.strip()
    else:
        salary = "N/A"

    # 工作描述
    job_description_element = soup.select("p.job-description__content")
    if job_description_element:
        job_description = " ".join([elem.text.strip() for elem in job_description_element])
    else:
      job_description = "N/A"

    # 工作經歷
    work_experience_element = soup.find("h3", text="工作經歷")
    if work_experience_element:
        work_experience_data = work_experience_element.find_next("div", class_="t3")
        work_experience = work_experience_data.text.strip() if work_experience_data else "N/A"
    else:
        work_experience = "N/A"

    # 學歷要求
    education_requirements_element = soup.find("h3", text="學歷要求")
    if education_requirements_element:
        education_data = education_requirements_element.find_next("div", class_="t3")
        education_requirements = education_data.text.strip() if education_data else "N/A"
    else:
        education_requirements = "N/A"

    # 擅長工具
    tools_element = soup.find("h3", text="擅長工具")
    if tools_element:
        tools_data = tools_element.find_next("div", class_="t3")
        tools = tools_data.text.strip() if tools_data else "N/A"
    else:
        tools = "N/A"

    # 工作技能
    skills_element = soup.find("h3", text="工作技能")
    if skills_element:
        skills_data = skills_element.find_next("div", class_="t3")
        skills = skills_data.text.strip() if skills_data else "N/A"
    else:
        skills = "N/A"

    # 其他條件
    other_conditions_element = soup.find("h3", text="其他條件")
    if other_conditions_element:
        other_conditions_data = other_conditions_element.find_next("p")
        other_conditions = other_conditions_data.text.strip() if other_conditions_data else "N/A"
    else:
        other_conditions = "N/A"

    # 將提取到的資料加入列表
    job_titles.append(job_title)
    employers.append(employer)
    job_locations.append(job_location)
    salaries.append(salary)
    job_descriptions.append(job_description)
    work_experience_list.append(work_experience)
    education_requirements_list.append(education_requirements)
    tools_list.append(tools)
    skills_list.append(skills)
    other_conditions_list.append(other_conditions)

    print(f"Processed job description for: {job_title}")

# 建立 DataFrame
df = pd.DataFrame({
    "job_title": job_titles,
    "employer": employers,
    "location": job_locations,
    "salary": salaries,
    "description": job_descriptions,
    "experience": work_experience_list,
    "education": education_requirements_list,
    "tools": tools_list,
    "skills": skills_list,
    "conditions": other_conditions_list
})


# 將資料保存為 CSV 檔案
df.to_csv("/content/job_data_n8n(0701).csv", index=False)
print("Data saved to job_data_n8n.csv")

Downloaded page 1 of job listings.
Downloaded page 2 of job listings.
Downloaded page 3 of job listings.
Downloaded page 4 of job listings.
Downloaded page 5 of job listings.
Downloaded page 6 of job listings.
Downloaded page 7 of job listings.
Downloaded page 8 of job listings.
Downloaded page 9 of job listings.
Downloaded page 10 of job listings.
Downloaded page 11 of job listings.
Downloaded page 12 of job listings.
Downloaded page 13 of job listings.
Downloaded page 14 of job listings.
Downloaded page 15 of job listings.
Downloaded page 16 of job listings.
Downloaded page 17 of job listings.
Downloaded page 18 of job listings.
Downloaded page 19 of job listings.
Downloaded page 20 of job listings.
Downloaded page 21 of job listings.
Downloaded page 22 of job listings.
Downloaded page 23 of job listings.
Downloaded page 24 of job listings.
Downloaded page 25 of job listings.
Downloaded page 26 of job listings.
Downloaded page 27 of job listings.
Downloaded page 28 of job listings.
D

/tmp/ipython-input-5-2211333262.py:99: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  work_experience_element = soup.find("h3", text="工作經歷")
/tmp/ipython-input-5-2211333262.py:107: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  education_requirements_element = soup.find("h3", text="學歷要求")
/tmp/ipython-input-5-2211333262.py:115: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tools_element = soup.find("h3", text="擅長工具")
/tmp/ipython-input-5-2211333262.py:123: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  skills_element = soup.find("h3", text="工作技能")
/tmp/ipython-input-5-2211333262.py:131: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  other_conditions_element = soup.find("h3", text="其他條件")


Processed job description for: AI生成服務專案應用經理
Processed job description for: AI影像設計
Processed job description for: 【知識】平台營運部｜策略聯盟經理 Affiliate Manager
Processed job description for: FAE & AI 應用規劃工程師  Field & AI application Engineer
Processed job description for: 【台北內科】Backend Engineer, AI & Legal Automation (Python/PHP) AI法律自動化後端工程師
Processed job description for: AI行銷企劃
Processed job description for: 視覺事業處_AI Agent & Vision Engineer (AI代理工程師)
Processed job description for: 數位轉型管理師
Processed job description for: HQ－（Sr.）數位轉型技術經理（汐止總部）
Processed job description for: Staff Engineer
Processed job description for: 自動化內容實習生｜AI 工具 /N8N
Processed job description for: 《市場營銷處》數據產品經理（Data Product Manager）｜網站營運 × 數據分析 × AI應用
Processed job description for: AI 整合顧問｜自動化流程設計 / 跨雲部署 / 生成式 AI 專案
Processed job description for: 【北士科】Full Stack Developer, AI LegalTech Platform (Python/PHP) AI法律科技平台全端工程師
Processed job description for: 【點點印】高階主管特助（溝通｜數據分析｜AI 工具）
Processed job description for: 雲端資安系統/助理工程師
Proce

In [ ]:
from google.colab import files
# 讓使用者上傳檔案
uploaded = files.upload()

# 取得檔案名稱
file_name = list(uploaded.keys())[0]

# 讀取 Excel 檔案
df = pd.read_excel(file_name) # 這裡使用您剛剛上傳的檔案名稱

TypeError: 'NoneType' object is not subscriptable

In [ ]:
font_path = "/content/NotoSansTC-Regular.ttf"
font = ImageFont.truetype(font_path, size=40)

# 創建一個空白圖像 (RGB 模式，寬 400 像素，高 200 像素，背景色為白色)
image = Image.new("RGB", (900, 200), (500, 200, 100))
draw = ImageDraw.Draw(image)

# 在圖像上繪製中文文字
text = "胡仁總灌君！"
draw.text((50, 50), text, font=font, fill=(0, 0, 0))

# 顯示圖像
plt.figure(figsize=(5, 5))
plt.imshow(image)
plt.axis('off')  # 關閉座標軸顯示
plt.show()

In [ ]:
# -------------------------------
# Step 1: 從 Excel 中讀取資料（假設 df 已經在環境中可用）
# -------------------------------
# 提取 description 和 conditions 欄位，並合併文字
def get_text_from_df(df):
    description_text = ' '.join(df['description'].dropna())
    skills_text = ' '.join(df['skills'].dropna())
    conditions_text = ' '.join(df['conditions'].dropna())
    tools_text = ' '.join(df[[f"tools.{i}" for i in range(1, 18)]].stack().astype(str))
    title_text = ' '.join(df['job_title'].dropna())
    return title_text
    #return tools_text

text = get_text_from_df(df)

# -------------------------------
# Step 2: 下載 Jieba 的繁體字典（若檔案不存在才下載）
# -------------------------------
dict_path = 'dict.txt.big'
if not os.path.exists(dict_path):
    !wget -O {dict_path} https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big

# -------------------------------
# Step 3: 設定 jieba 字典並進行斷詞
# -------------------------------
jieba.set_dictionary(dict_path)
seg_list = jieba.lcut(text, cut_all=False)

# -------------------------------
# Step 4: 關鍵詞抽取（取前 10 個）
# -------------------------------
keywords = jieba.analyse.extract_tags(text, topK=15, withWeight=False)
print(f"關鍵詞：{keywords}")

# -------------------------------
# Step 5: 使用 pandas 計算詞頻
# 定義停用詞集合，過濾無意義的詞彙（可依需求擴充）
# 定義連結詞變體（常見的及、與、和、等各種全形、半形形式）
linking_words_variants = {
    "及", "及,", "及，", "及.", "及．", "及…",
    ",及", "，及", "、及", ".及", "．及", "…及",
    "與", "與,", "與，", "與.", "與．", "與…",
    ",與", "，與", "、與", ".與", "．與", "…與",
    "和", "和,", "和，", "和.", "和．", "和…",
    ",和", "，和", "、和", ".和", "．和", "…和",
    "等", "等,", "等，", "等.", "等．", "等…",
    ",等", "，等", "、等", ".等", "．等", "…等",
}
stopwords = set([
    '，', ',', '。', '.', '、', '；', ';', '：', ':', '！', '!', '？', '?', '…', '⋯',
    '的', '了', '在', '是', '和', '及', '與', '或', '但', '而', '並', '且', '等',
    '這', '那', '一', '個', '一些', '每', '每個', '請', '可', '可能', '以上', '以下',
    '以及', '其中', '相關', '符合', '任職'
]) | linking_words_variants

def count_segment_freq(seg_list):
    seg_df = pd.DataFrame(seg_list, columns=['seg'])
    seg_df = seg_df[~seg_df['seg'].isin(stopwords)]
    seg_df['count'] = 1
    seg_freq = seg_df.groupby('seg')['count'].sum().sort_values(ascending=False)
    return seg_freq

seg_freq = count_segment_freq(seg_list)
print("詞頻統計前十五：")
print(seg_freq.head(15))

# 示範如何使用：
# 假設您已有斷詞結果 seg_list 是一個詞彙列表，且您知道當前處理的是哪個欄位（例如 job_title）
current_field = "job_title"  # 可依實際欄位調整
# 過濾掉該欄位的停用詞
#filtered_seg_list = [word for word in seg_list if word not in stopwords_by_field[current_field]]


# 您可以針對其他欄位類似地使用 stopwords_by_field 進行過濾


# -------------------------------
# Step 6: 檢查字體文件是否可用
# -------------------------------
# 此處使用您已確認正常的中文字體路徑
font_path = '/content/NotoSansTC-Regular.ttf'
try:
    font = ImageFont.truetype(font_path, size=20)
    print("字體加載成功")
except IOError:
    print("字體加載失敗，請檢查文件是否正確")
    raise



In [ ]:
我做一個吃貪吃 # -------------------------------
# Step 7: 生成文字雲
# -------------------------------
wc = WordCloud(
    font_path=font_path,          # 指定中文字體
    background_color="white",     # 背景顏色
    max_words=100,                # 最大顯示詞數
    width=1000,                    # 圖片寬度
    height=500                    # 圖片高度
)

# 將斷詞結果轉換為以空格分隔的字符串
seg_text = ' '.join(seg_list)

if seg_text.strip():
    wc.generate(seg_text)
    plt.figure(figsize=(40, 20),dpi=500)
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.show()
else:
    print("斷詞結果為空，無法生成文字雲")